In [1]:
import librosa as lr
import numpy as np
import scipy.signal as signal
import soundfile as sf
import matplotlib.pyplot as plt
from scipy.io import wavfile
import IPython.display as ipd
from AudioUtilsCustom import *




In [2]:
from compression_custom import Audiopackage
from compression_custom import AudioSegment
import time


In [3]:
# filepath = 'C:\\Users\\chorj\\Downloads\\WavFiles'
# input_file1 = filepath + '\\Hercules_dry_Drums_Kick.wav'
# input_file2 = filepath + '\\Hercules_dry_Drums_Loop.wav'

import os
filepath = 'data/Hercules_Dry(AbelEdit)'
input_file1 = os.path.join(filepath, 'Hercules_dry_Drums_Kick.wav')
input_file2 = os.path.join(filepath, 'Hercules_dry_Drums_Loop.wav')

# Read the wav file 
#samplerate1, data1 = wavfile.read(input_file1)
#samplerate2, data2 = wavfile.read(input_file2)
#print('Sampling rate = ', samplerate1)
#dt1, sr1 = lr.load(input_file1, sr = samplerate1)
#dt2, sr2 = lr.load(input_file2, sr = samplerate2)


In [5]:
#TODO: 각 시도마다 초기화가 완전히 안 되어서 time measurement가 정확하지 않음.
start_time = time.time()
example = Audiopackage(input_file1, "exact")
newtime = time.time() - start_time
print("Seconds taken: " + str(newtime))
start_time = time.time()
example2 = Audiopackage(input_file2, "exact")
newtime2 = time.time() - start_time
print("Seconds taken: " + str(newtime2))
start_time = time.time()
example3 = Audiopackage(input_file1, "similar")
newtime3 = time.time() - start_time
print("Seconds taken: " + str(newtime3))
start_time = time.time()
example4 = Audiopackage(input_file2, "similar")
newtime4 = time.time() - start_time
print("Seconds taken: " + str(newtime4))
start_time = time.time()
example5 = Audiopackage(input_file1, "spectrogram")
newtime5 = time.time() - start_time
print("Seconds taken: " + str(newtime5))
start_time = time.time()
example6 = Audiopackage(input_file2, "spectrogram")
newtime6 = time.time() - start_time
print("Seconds taken: " + str(newtime6))

Seconds taken: 1.176893949508667
Seconds taken: 0.7234981060028076
Seconds taken: 13.345046997070312


/Users/hyeonseokc/Desktop/GHCho/Multitrack-Compress/compression_custom.py:10: WavFileWarning: Chunk (non-data) not understood, skipping it.
  samplerate, audio = wavfile.read(audiofile)


Seconds taken: 8.709228992462158


/Users/hyeonseokc/Desktop/GHCho/Multitrack-Compress/compression_custom.py:10: WavFileWarning: Chunk (non-data) not understood, skipping it.
  samplerate, audio = wavfile.read(audiofile)


Seconds taken: 0.8747153282165527
Seconds taken: 0.8287391662597656


/Users/hyeonseokc/Library/Python/3.9/lib/python/site-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=512 is too large for input signal of length=416
  warnings.warn(


In [5]:
example.printInfo()
example2.printInfo()
example3.printInfo()
example4.printInfo()
example5.printInfo()
example6.printInfo()

Audio Nane:  Hercules_dry_Drums_Kick.wav
Audio Length:  5534833
Segment Size:  33
Sum of Segments:  1403368
Ratio of Compression:  0.25355200418874424
Audio Nane:  Hercules_dry_Drums_Loop.wav
Audio Length:  5534833
Segment Size:  77
Sum of Segments:  3528482
Ratio of Compression:  0.6375046907467669
Audio Nane:  Hercules_dry_Drums_Kick.wav
Audio Length:  5534833
Segment Size:  15
Sum of Segments:  1077239
Ratio of Compression:  0.19462899783968188
Audio Nane:  Hercules_dry_Drums_Loop.wav
Audio Length:  5534833
Segment Size:  46
Sum of Segments:  3221101
Ratio of Compression:  0.5819689591357138
Audio Nane:  Hercules_dry_Drums_Kick.wav
Audio Length:  5534833
Segment Size:  15
Sum of Segments:  1077239
Ratio of Compression:  0.19462899783968188
Audio Nane:  Hercules_dry_Drums_Loop.wav
Audio Length:  5534833
Segment Size:  32
Sum of Segments:  3113021
Ratio of Compression:  0.5624417213672029


In [6]:
def find_non_zero_ratio(sample):
    count = 0
    for i in range(len(sample)):
        if sample[i][0] == 0 and sample[i][1] == 0:
            count += 1
    return round(1 - count/len(sample),4)


In [7]:
import soundfile as sf
import zipfile
import os
import time

def compare_compression_ratio(input_file):
    # Specify the output file paths
    flac_output_file = 'compressed.flac'
    zip_output_file = 'compressed.zip'

    # Read the WAV file
    data, samplerate = sf.read(input_file)

    start_time = time.time()
    # Write the FLAC file
    sf.write(flac_output_file, data, samplerate, format='flac')
    # time
    flac_time = time.time() - start_time

    start_time = time.time()
    # Create a new ZIP file
    with zipfile.ZipFile(zip_output_file, 'w', compression=zipfile.ZIP_DEFLATED) as zipf:
        # Add the WAV file to the ZIP file
        zipf.write(input_file, arcname='audio.wav')
    # time
    zip_time = time.time() - start_time

    # Get the file sizes
    input_size = os.path.getsize(input_file)
    flac_size = os.path.getsize(flac_output_file)
    zip_size = os.path.getsize(zip_output_file)

    start_time = time.time()
    tryExact = Audiopackage(input_file, "exact")
    exact_time = time.time() - start_time

    start_time = time.time()
    trySimilar = Audiopackage(input_file, "similar")
    similar_time = time.time() - start_time

    start_time = time.time()
    trySpectrogram = Audiopackage(input_file, "spectrogram")
    spectrogram_time = time.time() - start_time
    
    spectrogram_ratio = trySpectrogram.compratio()
    similar_ratio = trySimilar.compratio()
    exact_ratio = tryExact.compratio()
    flac_ratio = flac_size / input_size
    zip_ratio = zip_size / input_size
    ratiofamily = [zip_ratio, flac_ratio, exact_ratio, similar_ratio, spectrogram_ratio]
    timefamily = [zip_time, flac_time, exact_time, similar_time, spectrogram_time]
    non_zero_ratio = find_non_zero_ratio(data)
    baseline = non_zero_ratio
    
    # Calculate the compression ratios
   
    # Delete the temporary files
    os.remove(flac_output_file)
    os.remove(zip_output_file)
    return [ratiofamily, timefamily, baseline]


In [8]:
def resultanalysis(ratiofamily, timefamily, baseline):
    print("Baseline: " + str(baseline))
    print("ZIP Compression Ratio: " + str(ratiofamily[0]))
    print("FLAC Compression Ratio: " + str(ratiofamily[1]))
    print("Exact Compression Ratio: " + str(ratiofamily[2]))
    print("Similar Compression Ratio: " + str(ratiofamily[3]))
    print("Spectrogram Compression Ratio: " + str(ratiofamily[4]))
    print("ZIP Compression Time: " + str(timefamily[0]))
    print("FLAC Compression Time: " + str(timefamily[1]))
    print("Exact Compression Time: " + str(timefamily[2]))
    print("Similar Compression Time: " + str(timefamily[3]))
    print("Spectrogram Compression Time: " + str(timefamily[4]))

a = compare_compression_ratio(input_file1)
resultanalysis(a[0], a[1], a[2])

c:\Users\chorj\Documents\GitHub\Multitrack-Compress\compression_custom.py:10: WavFileWarning: Chunk (non-data) not understood, skipping it.
  samplerate, audio = wavfile.read(audiofile)


Baseline: 0.4512
ZIP Compression Ratio: 0.36612599549920144
FLAC Compression Ratio: 0.13914049406402765
Exact Compression Ratio: 0.25355200418874424
Similar Compression Ratio: 0.19462899783968188
Spectrogram Compression Ratio: 0.19462899783968188
ZIP Compression Time: 0.3965573310852051
FLAC Compression Time: 0.17219066619873047
Exact Compression Time: 1.2862205505371094
Similar Compression Time: 13.789097547531128
Spectrogram Compression Time: 0.9885334968566895


In [9]:
a = compare_compression_ratio(input_file2)
resultanalysis(a[0], a[1], a[2])

c:\Users\chorj\Documents\GitHub\Multitrack-Compress\compression_custom.py:10: WavFileWarning: Chunk (non-data) not understood, skipping it.
  samplerate, audio = wavfile.read(audiofile)
C:\Users\chorj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1115
  warnings.warn(
C:\Users\chorj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1103
  warnings.warn(
C:\Users\chorj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=845
  warnings.warn(
C:\Users\chorj\AppData\Local\Packages\PythonSoftwa

Baseline: 0.6603
ZIP Compression Ratio: 0.5855697053751763
FLAC Compression Ratio: 0.24142269775819472
Exact Compression Ratio: 0.6375046907467669
Similar Compression Ratio: 0.5819689591357138
Spectrogram Compression Ratio: 0.5624417213672029
ZIP Compression Time: 0.7502789497375488
FLAC Compression Time: 0.20037245750427246
Exact Compression Time: 0.7628915309906006
Similar Compression Time: 9.545132160186768
Spectrogram Compression Time: 0.9170928001403809


In [10]:
#여기부터는 개별 파일이 아닌 폴더를 대상으로 압축률을 측정하는 코드

import glob

# Specify the folder path
folder_path = 'C:\\Users\\chorj\\Downloads\\WavFiles\\Hercules_Dry(AbelEdit)'

# Find all WAV files in the folder
wav_files = glob.glob(folder_path + '/*.wav')

# Print the list of WAV files
for file in wav_files:
    print(file)


C:\Users\chorj\Downloads\WavFiles\Hercules_Dry(AbelEdit)\Hercules_dry_Bass_BassTop.wav
C:\Users\chorj\Downloads\WavFiles\Hercules_Dry(AbelEdit)\Hercules_dry_Bass_Sub.wav
C:\Users\chorj\Downloads\WavFiles\Hercules_Dry(AbelEdit)\Hercules_dry_Brass_Horns.wav
C:\Users\chorj\Downloads\WavFiles\Hercules_Dry(AbelEdit)\Hercules_dry_Brass_Low Brs.wav
C:\Users\chorj\Downloads\WavFiles\Hercules_Dry(AbelEdit)\Hercules_dry_Brass_Trombone.wav
C:\Users\chorj\Downloads\WavFiles\Hercules_Dry(AbelEdit)\Hercules_dry_Brass_Trumpet.wav
C:\Users\chorj\Downloads\WavFiles\Hercules_Dry(AbelEdit)\Hercules_dry_Choir_Men.wav
C:\Users\chorj\Downloads\WavFiles\Hercules_Dry(AbelEdit)\Hercules_dry_Choir_Women.wav
C:\Users\chorj\Downloads\WavFiles\Hercules_Dry(AbelEdit)\Hercules_dry_Drums)Crash Body.wav
C:\Users\chorj\Downloads\WavFiles\Hercules_Dry(AbelEdit)\Hercules_dry_Drums_Crash.wav
C:\Users\chorj\Downloads\WavFiles\Hercules_Dry(AbelEdit)\Hercules_dry_Drums_Kick.wav
C:\Users\chorj\Downloads\WavFiles\Hercules_Dry(

In [ ]:
def analyze_folder(folder_path):
    # Find all WAV files in the folder
    wav_files = glob.glob(folder_path + '/*.wav')
    resultList = []
    # Print the list of WAV files
    for file in wav_files:
        print(file)
        resultList.append(compare_compression_ratio(file))
    return [resultList]


In [ ]:
resultList = analyze_folder(folder_path)    

C:\Users\chorj\Downloads\WavFiles\Hercules_Dry(AbelEdit)\Hercules_dry_Bass_BassTop.wav
Ratio of non-zero: 0.73
FLAC Compression Ratio: 0.0889
ZIP Compression Ratio: 0.5792
C:\Users\chorj\Downloads\WavFiles\Hercules_Dry(AbelEdit)\Hercules_dry_Bass_Sub.wav
Ratio of non-zero: 0.7873
FLAC Compression Ratio: 0.0577
ZIP Compression Ratio: 0.7135
C:\Users\chorj\Downloads\WavFiles\Hercules_Dry(AbelEdit)\Hercules_dry_Brass_Horns.wav
Ratio of non-zero: 0.8507
FLAC Compression Ratio: 0.1985
ZIP Compression Ratio: 0.7520
C:\Users\chorj\Downloads\WavFiles\Hercules_Dry(AbelEdit)\Hercules_dry_Brass_Low Brs.wav
Ratio of non-zero: 0.8055
FLAC Compression Ratio: 0.1624
ZIP Compression Ratio: 0.7061
C:\Users\chorj\Downloads\WavFiles\Hercules_Dry(AbelEdit)\Hercules_dry_Brass_Trombone.wav
Ratio of non-zero: 0.1763
FLAC Compression Ratio: 0.0406
ZIP Compression Ratio: 0.1430
C:\Users\chorj\Downloads\WavFiles\Hercules_Dry(AbelEdit)\Hercules_dry_Brass_Trumpet.wav
Ratio of non-zero: 0.7558
FLAC Compression Rat

In [ ]:
non_zero_ratio = 0
flac_ratio = 0
zip_ratio = 0
for i in range(len(resultList)):
    non_zero_ratio += float(resultList[i][0])
    flac_ratio += float(resultList[i][1])
    zip_ratio += float(resultList[i][2])

print("Average non-zero ratio: " + str(non_zero_ratio/len(resultList)))
print("Average flac ratio: " + str(flac_ratio/len(resultList)))
print("Average zip ratio: " + str(zip_ratio/len(resultList)))

print("Zip to non-zero Compression ratio: " + str(zip_ratio/non_zero_ratio))
print("FLAC to non-zero Compression ratio: " + str(flac_ratio/non_zero_ratio))
print("Zip to FLAC Compression ratio: " + str(flac_ratio/zip_ratio))

Average non-zero ratio: 0.75865
Average flac ratio: 0.07329174293944803
Average zip ratio: 0.6463672717099354
Zip to non-zero Compressoin ratio: 0.8519966673827658
FLAC to non-zero Compressoin ratio: 0.09660811037955318
Zip to FLAC Compressoin ratio: 0.11339024444346948
